In [ ]:
import pyterrier as pt
pt.init()

In [ ]:
SOURCEDIR="/Users/craigm/Downloads/GoldSetOfDuplicates"

In [ ]:
def makeHash(corpus_files, mhs):
    Arrays = pt.autoclass("java.util.Arrays")
    corpus_file_list = Arrays.asList(corpus_files)
    print(corpus_file_list.toString())
    trec_properties = {
        "TrecDocTags.doctag": "DOC",
        "TrecDocTags.idtag": "DOCNO",
        "TrecDocTags.skip": "DOCHDR",
        "TrecDocTags.casesensitive": "false",
        "trec.collection.class": "TRECCollection",
        "indexer.meta.forward.keylens": "200"
    }
    for k,v in trec_properties.items():
        pt.ApplicationSetup.setProperty(k, v)
    corpus = pt.autoclass("org.terrier.indexing.SimpleFileCollection")(
        corpus_file_list, 
        False)

    def _get_text(d):
        terms =[]
        while not d.endOfDocument():
            t = d.getNextTerm()
            if t is None:
                continue
            terms.append(t)
        return " ".join(terms)

    def _corpus_iter():
        while corpus.nextDocument():
            doc = corpus.getDocument()
            if doc is None:
                continue
            text = _get_text(doc)
            docno = doc.getProperty("filename")
            print(docno)
            yield docno, text

    mhs.index(_corpus_iter())
    return mhs

In [ ]:
from pyterrier_neardup.shingler import MinHashShingler
mhs = makeHash(SOURCEDIR, MinHashShingler())


In [ ]:
queryDocnos = [ docno for docno in pt.Utils.get_files_in_dir(SOURCEDIR) if "original" in docno]
queryDocnos

In [ ]:
import pandas as pd, numpy as np
queries = pd.DataFrame(np.array(queryDocnos).reshape(-1,1), columns=["docno"])
queries

In [ ]:
def get_res(mhs, queries):
    df = mhs.transform(queries)
    df = df.rename(columns={"docno_x" : "qid", "docno_y" : "docno"})[["qid", "docno", "score"]]
    df = pt.model.add_ranks(df)
    return df
get_res(mhs, queries)

In [ ]:
mhs.sim_threshold = 0.6

In [ ]:
from collections import defaultdict
qrels = defaultdict(dict)
import os
for file in pt.Utils.get_files_in_dir(SOURCEDIR):
    dir = os.path.dirname(file)
    base = os.path.basename(file)
    if base == "original.htm":
        continue
    qrels[dir + "/original.htm"][file] = 1
#qrels


In [ ]:
scan={}
from tqdm import tqdm
for t in tqdm( list(x / 10 for x in range(0,10)) ):
    mhs.sim_threshold = t
    scan[t] = pt.Utils.evaluate(get_res(mhs, queries), qrels, ["set_F"]).get("set_F", 0)

In [ ]:
scan

In [ ]:
from pyterrier_neardup.snapy_lsh import SnapyMinHash
pt.Utils.evaluate(get_res(makeHash(SOURCEDIR, SnapyMinHash()), queries), qrels, ["set_F"]).get("set_F", 0)